In [7]:
#deploy ONNX model
import onnx.helper
N_CLASSES = 1000 # Our ResNet-50 is trained on a 1000 class ImageNet task
trt_model = ONNXClassifierWrapper("resnet_engine.trt", [BATCH_SIZE, N_CLASSES], target_dtype = PRECISION)

NameError: name 'ONNXClassifierWrapper' is not defined

In [9]:
#ONNX conversion and deployment
from tensorflow.keras.applications import ResNet50

model = ResNet50(weights = 'imagenet')

102967424/102967424 [==============================] - 13s 0us/step


In [ ]:
#convert Resnet50 from h5 to onnx_model
import tf2onnx

model.save('my_model')
!python -m tf2onnx.convert --saved-model my_model --output temp.onnx
onnx_model = onnx.load_model('temp.onnx')

In [14]:
#Set an explicit batch size in the ONNX file
import onnx
BATCH_SIZE = 64
inputs = onnx_model.graph.input
for input in inputs:
    dim1 = input.type.tensor_type.shape.dim[0]
    dim1.dim_value = BATCH_SIZE

In [15]:
#Save the ONNX file
model_name = "resnet50_onnx_model.onnx"
onnx.save_model(onnx_model, model_name)

In [16]:
#convert pytorch to ONNX

#import resnet model
import torchvision.models as models

resnext50_32x4d = models.resnext50_32x4d(pretrained=True)

#save ONNX file from pytorch
import torch

BATCH_SIZE = 64
dummy_input=torch.randn(BATCH_SIZE, 3, 224, 224)

#Save ONNX file
import torch.onnx
torch.onnx.export(resnext50_32x4d, dummy_input, "resnet50_onnx_model(from Pytorch).onnx", verbose=False)

c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to C:\Users\PC/.cache\torch\hub\checkpoints\resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

In [19]:
#convert ONNX to TensorRT
!trtexec --onnx=resnet50_onnx_model.onnx --saveEngine=resnet_engine(pytorch).trt

&&&& RUNNING TensorRT.trtexec [TensorRT v8402] # trtexec --onnx=resnet50_onnx_model.onnx --saveEngine=resnet_engine(pytorch).trt
[10/31/2022-21:03:46] [I] === Model Options ===
[10/31/2022-21:03:46] [I] Format: ONNX
[10/31/2022-21:03:46] [I] Model: resnet50_onnx_model.onnx
[10/31/2022-21:03:46] [I] Output:
[10/31/2022-21:03:46] [I] === Build Options ===
[10/31/2022-21:03:46] [I] Max batch: explicit batch
[10/31/2022-21:03:46] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[10/31/2022-21:03:46] [I] minTiming: 1
[10/31/2022-21:03:46] [I] avgTiming: 8
[10/31/2022-21:03:46] [I] Precision: FP32
[10/31/2022-21:03:46] [I] LayerPrecisions: 
[10/31/2022-21:03:46] [I] Calibration: 
[10/31/2022-21:03:46] [I] Refit: Disabled
[10/31/2022-21:03:46] [I] Sparsity: Disabled
[10/31/2022-21:03:46] [I] Safe mode: Disabled
[10/31/2022-21:03:46] [I] DirectIO mode: Disabled
[10/31/2022-21:03:46] [I] Restricted mode: Disabled
[10/31/2022-21:03:46] [I] Bui

[10/31/2022-21:03:48] [W] [TRT] onnx2trt_utils.cpp:369: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[10/31/2022-21:04:20] [W] [TRT] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[10/31/2022-21:04:20] [W] [TRT] The getMaxBatchSize() function should not be used with an engine built from a network created with NetworkDefinitionCreationFlag::kEXPLICIT_BATCH flag. This function will always return 1.
[10/31/2022-21:04:24] [W] * GPU compute time is unstable, with coefficient of variance = 3.64305%.
[10/31/2022-21:04:24] [W]   If not already in use, locking GPU clock frequency or adding --useSpinWait may improve the stability.
